In [1]:
import numpy as np
import os
import pandas as pd

import matplotlib
# For draw in bg
matplotlib.use('Agg')
import matplotlib.pyplot as plt
# For draw in bg
plt.switch_backend('agg')

from scipy.signal import medfilt
matplotlib.rcParams.update({'font.size': 8})

import seaborn as sns
sns.set(style="darkgrid")

from visdom import Visdom

INF = 1e9

In [2]:
def plot(viz, win, dirname, targs_names, quant=2, smooth=4, opt='each'):
    
    if opt == 'each': 
        data, y_name = load_data(dirname, targs_names, smooth_max=smooth)        
    elif opt == 'best': 
        data, y_name = load_data_with_transform_best(dirname, targs_names, quant=quant, smooth_max=smooth)

    sns_plot = sns.relplot(x="Epoch", y=y_name, hue='type', kind="line", data=data)
    fig = sns_plot.fig
    plt.title(dirname)
    figname = os.path.join(dirname, 'eval_result.png')
    fig.savefig(figname)
    image = plt.imread(figname)

    # Show it in visdom
    image = np.transpose(image, (2, 0, 1))    
    return viz.image(image, win=win)
def load_data(dirname, targs_names, smooth_max=4):
    df = pd.read_csv(os.path.join(dirname, 'progress.csv'))

    targs = (list(targs_names.keys()))
    df = df[df[targs].notnull()]
    
    smooth_range = 1 if len(df) <= 20 else smooth_max
    
    plot_data = [] 
    for targ, name in sorted(targs_names.items()):
        d = df[[targ]].copy(deep=True)
        d['Epoch'] = range(0, len(d))
        d = d.rolling(smooth_range, center=True, min_periods=1).mean()
        d['Epoch'] = d['Epoch'].astype(int)
        d.insert(2, 'type', name)
        d = d.rename(index=str, columns={targ: 'Reward'})
        plot_data.append(d)    
    df = pd.concat(plot_data)
    return df, 'Reward'

In [3]:
df=pd.read_csv(os.path.join('/Users/zsbjltwjj/Desktop/logs/nosil', 'progress.csv'))

EmptyDataError: No columns to parse from file

In [25]:
df['MeanRewMF'].dropna(how="all")

4    18.588898
9    48.251260
Name: MeanRewMF, dtype: float64

In [47]:
from baselines.common import plot_util as pu
results = pu.load_results('~/logs/cartpole-ppo') 
for r in results:
    df=r.progress.copy(deep=True)
    df.insert(0,"Best",0.0)
    max_reward_so_far = -INF
    for idx, row in df.iterrows():            
        max_reward_so_far = max(row['AverageReturn'], max_reward_so_far)
        df.at[idx, 'Best']=max_reward_so_far
    print(df)

        Best  AverageReturn  eplenmean  eprewmean   fps  loss/approxkl  \
0   15.37500       15.37500   15.37500   15.37500   390   9.459308e-07   
1   20.95082       20.95082   20.95082   20.95082  2053   2.929393e-05   
2   24.77000       24.77000   24.77000   24.77000  2046   4.829291e-05   
3   29.17000       29.17000   29.17000   29.17000  2069   1.373136e-04   
4   31.11000       31.11000   31.11000   31.11000  2038   1.099520e-05   
5   31.11000       30.97000   30.97000   30.97000  2065   8.766213e-04   
6   31.11000       29.94000   29.94000   29.94000  2092   7.025587e-05   
7   31.11000       27.00000   27.00000   27.00000  2051   7.268270e-04   
8   31.11000       29.60000   29.60000   29.60000  2107   3.599036e-05   
9   31.11000       30.40000   30.40000   30.40000  2094   7.955600e-05   
10  34.13000       34.13000   34.13000   34.13000  2069   1.825759e-05   
11  34.13000       33.83000   33.83000   33.83000  2079   2.631606e-05   
12  38.81000       38.81000   38.81000

In [61]:
b=df['Best'].copy(deep=True)
a=(r.progress['misc/total_timesteps']/1000).copy(deep=True)

In [70]:
b[b.index % 4 ==0]
a[b.index % 4 ==0].index

Int64Index([0, 4, 8, 12, 16, 20], dtype='int64')

In [13]:
from baselines.common import plot_util as pu
results = pu.load_results('~/logs/cartpole-ppo') 
r=results[0]
df=r.progress
df[df['AverageReturn'].notnull()]

,AverageReturn,eplenmean,eprewmean,fps,loss/approxkl,loss/clipfrac,loss/policy_entropy,loss/policy_loss,loss/value_loss,misc/explained_variance,misc/nupdates,misc/serial_timesteps,misc/time_elapsed,misc/total_timesteps
0,15.37500,15.37500,15.37500,390,9.459308e-07,0.0,0.693142,0.000054,24.012745,0.074818,1,128,0.327920,128
1,20.95082,20.95082,20.95082,2053,2.929393e-05,0.0,0.687419,-0.000985,42.580452,-0.004958,10,1280,0.908423,1280
2,24.77000,24.77000,24.77000,2046,4.829291e-05,0.0,0.684248,-0.000854,48.439617,0.128270,20,2560,1.526412,2560
3,29.17000,29.17000,29.17000,2069,1.373136e-04,0.0,0.672147,-0.000729,16.727566,0.099008,30,3840,2.153118,3840
4,31.11000,31.11000,31.11000,2038,1.099520e-05,0.0,0.668190,0.000075,19.776640,0.075361,40,5120,2.774607,5120
5,30.97000,30.97000,30.97000,2065,8.766213e-04,0.0,0.686000,-0.001568,27.794363,0.030894,50,6400,3.390006,6400
6,29.94000,29.94000,29.94000,2092,7.025587e-05,0.0,0.683445,-0.001122,23.645378,0.069670,60,7680,4.006954,7680
7,27.00000,27.00000,27.00000,2051,7.268270e-04,0.0,0.640873,-0.002434,15.259211,0.210977,70,8960,4.625229,8960
8,29.60000,29.60000,29.60000,2107,3.599036e-05,0.0,0.672401,-0.000135,21.358757,0.114085,80,10240,5.239102,10240
9,30.40000,30.40000,30.40000,2094,7.955600e-05,0.0,0.660324,-0.000901,30.239300,0.123040,90,11520,5.856453,11520
